<a href="https://colab.research.google.com/github/haterzd/Neural-Networks/blob/main/text%20segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pymorphy2 # установка лемматизатора pymorphy2

     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 8.2 MB 11.7 MB/s 


In [ ]:
!pip install tensorflow==2.3 # устанавливаем версию tf 2.3 и делаем restart runtime

In [ ]:
import numpy as np # Библиотека для работы с массивами данных
import pandas as pd
from tensorflow.keras.models import Model, load_model # Импортируем Model, load_model - метод, что загружает предобученную сеть
import re # Имортируем чтобы работать с строками
from tensorflow.keras.preprocessing.text import Tokenizer # Метод, который поволяет работать с текстами и конвертирует их в последовательности (индексов)
# Импорт слоёв нейронных сетей
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Conv1DTranspose, Lambda
from tensorflow.keras import backend as K # Импортируем, чтобы высчитать dice_coef(ошибку)
from tensorflow.keras.optimizers import Adam, Adadelta # Импортируем оптимизаторы
from tensorflow.keras import utils # Импортируем для работы с категориальными данными
import tensorflow
from google.colab import files # Импорт для работы с файлами
import matplotlib.pyplot as plt # Импорт для отрисовывания графиков
from gensim.models import word2vec # Импортируем gensim
import os # Импортируем для работы с системными файлами
import pandas as pd # Импортируем для работы с Массивами данных(таблицами - Датафреймами)
import time # Имортируем, чтобы высчитать время работы каких-либо процессов
import nltk #Natural language toolkit - Инструментарий естественного языка
from nltk.stem import WordNetLemmatizer  # Импортируем для работы с леммами
import pymorphy2 # Импортируем для работы с леммами
import spacy # Импортируем для работы с леммами на английском языке

nltk.download('wordnet') # Скачиваем сетку слов для лемматизации

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/NER_Dataset.csv')

In [ ]:
df.head()

,Sentence_ID,Word,POS,Tag
0,Sentence: 1,"['Thousands', 'of', 'demonstrators', 'have', '...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 10,"['Iranian', 'officials', 'say', 'they', 'expec...","['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
2,Sentence: 100,"['Helicopter', 'gunships', 'Saturday', 'pounde...","['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...","['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '..."
3,Sentence: 1000,"['They', 'left', 'after', 'a', 'tense', 'hour-...","['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 10000,"['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...","['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...","['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47959 entries, 0 to 47958
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sentence_ID  47959 non-null  object
 1   Word         47959 non-null  object
 2   POS          47959 non-null  object
 3   Tag          47959 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [ ]:
df.Tag[1]

['B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-tim',
 'O',
 'O',
 'O',
 'B-org',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
for i in range(47959):
    df.Word[i] = df.Word[i].split(', ')
    df.Tag[i] = df.Tag[i].split(',')

In [ ]:
del_symbols_for_word = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";",'•','%', ' ', '[', ']', '\'', ',,', '\"']
del_symbols_for_tag = ['\'', ']', '[', ' ']
for i in range(47959):
    for j in range(len(df.Word[i])):
        for k in del_symbols_for_word:
            if k in df.Word[i][j]:
                df.Word[i][j] = df.Word[i][j].replace(k, '')
        for l in del_symbols_for_tag:
            if l in df.Tag[i][j]:
                df.Tag[i][j] = df.Tag[i][j].replace(l, '')

In [ ]:
list_of_tags = []
for i in range(47959):
    for j in range(len(df.Tag[i])):
        if df.Tag[i][j] not in list_of_tags:
            list_of_tags.append(df.Tag[i][j])
print(list_of_tags)

['O', 'B-geo', 'B-gpe', 'B-tim', 'B-org', 'I-geo', 'B-per', 'I-per', 'I-org', 'I-tim', 'B-art', 'I-art', 'B-nat', 'I-gpe', 'I-nat', 'B-eve', 'I-eve']


In [ ]:
q = 0
for i in range(47959):
    for j in range(len(df.Tag[i])):
        if df.Tag[i][j] == list_of_tags[1]:
            print(df.Word[i][j])
            q += 1
            if q == 10:
                break
    if q == 10:
        break

London
Iraq
Orakzai
South
UN
US
Aceh
Maldives
India
Asia


In [ ]:
for i in range(len(df.Word)):
    while '' in df.Word[i]:
        space_index = df.Word[i].index('')
        del df.Word[i][space_index]
        del df.Tag[i][space_index] 

In [ ]:
print(df.Word[555])
print(len(df.Word[555]))

['She', 'is', 'one', 'of', 'this', 'year', 's', 'recipients', 'of', 'the', 'European', 'Parliament', 's', 'Sakharov', 'prize', 'for', 'freedom', 'of', 'thought']
19


In [ ]:
print(df.Tag[555])
print(len(df.Tag[555]))

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'I-org', 'I-org', 'O', 'O', 'O', 'O', 'O']
19


In [ ]:
q = 0
for i in range(47959):
    if len(df.Word[i]) != len(df.Tag[i]):
        q += 1
print(q)

0


In [ ]:
' '.join(df.Word[0])

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country'

In [ ]:
nlp = spacy.load('en')

In [ ]:
spacy_results = nlp(' '.join(df.Word[0]))

In [ ]:
[token.lemma_ for token in spacy_results]

['thousand',
 'of',
 'demonstrator',
 'have',
 'march',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'british',
 'troop',
 'from',
 'that',
 'country']

In [ ]:
X = []
for i in range(len(df.Word)):
    spacy_results = nlp(' '.join(df.Word[i]).lower())
    X.append([token.lemma_ for token in spacy_results])

In [ ]:
X[20]

['doctor',
 'say',
 'the',
 'stroke',
 'have',
 'not',
 'cause',
 'any',
 'permanent',
 'damage']

In [ ]:
len(X)

47959

In [ ]:
list_of_tags = np.array(list_of_tags)

In [ ]:
y_data = []
X_data = []
num_of_cat = len(list_of_tags)
for i in range(len(df.Tag)):
    for j in range(len(df.Tag[i])):
        index = np.argwhere(df.Tag[i][j] == list_of_tags)[0][0]
        y_data.append(utils.to_categorical(index, num_of_cat))
        X_data.append(X[i][j])

In [ ]:
tokenizer = Tokenizer(lower=True, filters = '', char_level=False)


tokenizer.fit_on_texts(X_data) # Скармливаем тексты токенайзеру
clean_voc = {}                    # Создаем пустой словарь

for item in tokenizer.word_index.items(): # Превращаем лист в словарь
  clean_voc[item[0]] = item[1]     

In [ ]:
print('Словарь все слов по их частотноти:') 
print(clean_voc, '\n')
print('Длина словаря:', len(clean_voc))


Словарь все слов по их частотноти:
{'the': 1, 'in': 2, 'of': 3, 'be': 4, 'to': 5, 'a': 6, '-pron-': 7, 'and': 8, 'say': 9, 'have': 10, 's': 11, 'for': 12, 'on': 13, 'that': 14, 'with': 15, 'at': 16, 'from': 17, 'by': 18, 'an': 19, 'as': 20, 'official': 21, 'us': 22, 'president': 23, 'kill': 24, 'will': 25, 'government': 26, 'mr': 27, 'two': 28, 'people': 29, 'country': 30, 'after': 31, 'not': 32, 'year': 33, 'but': 34, 'more': 35, 'attack': 36, 'also': 37, 'new': 38, 'other': 39, 'report': 40, 'minister': 41, 'united': 42, 'last': 43, 'military': 44, 'who': 45, 'force': 46, 'than': 47, 'police': 48, 'group': 49, 'iraq': 50, 'one': 51, 'week': 52, 'which': 53, 'security': 54, 'this': 55, 'call': 56, 'leader': 57, 'about': 58, 'militant': 59, 'month': 60, 'least': 61, 'state': 62, 'election': 63, 'tuesday': 64, 'take': 65, 'states': 66, 'since': 67, 'world': 68, 'authority': 69, 'thursday': 70, 'include': 71, 'iran': 72, 'over': 73, 'friday': 74, 'monday': 75, 'city': 76, 'wednesday': 77

In [ ]:
len(clean_voc)

26235

In [ ]:
a = np.array([1, 2, 5, 34, 432])
np.argwhere(a == 5)

array([[2]])

In [ ]:
print(len(y_data))
print(len(X_data))

960619
960619


In [ ]:
X_train = tokenizer.texts_to_sequences(X_data[:-50000])
X_test = tokenizer.texts_to_sequences(X_data[-50000:])

In [ ]:
y_train = y_data[:-50000]
y_test = y_data[-50000:]

In [ ]:
X_train[:10]

[[233], [3], [974], [10], [321], [253], [537], [5], [207], [1]]

In [ ]:
for i in range(len(X_train)):
    X_train[i] = X_train[i][0]

for i in range(len(X_test)):
    X_test[i] = X_test[i][0]

In [ ]:
X_train[:10]

[233, 3, 974, 10, 321, 253, 537, 5, 207, 1]

In [ ]:
X_data[:10]

['thousand',
 'of',
 'demonstrator',
 'have',
 'march',
 'through',
 'london',
 'to',
 'protest',
 'the']

In [ ]:
y_data[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [ ]:
# Создание выборки из индексов
def getSetFromIndices(wordIndices, xLen, step): 
  xBatch = [] # Лист, хранящий фрагменты текста
  wordsLen = len(wordIndices) # получаем длину текста
  index = 0 #  стартовый индекс
  
  while (index + xLen <= wordsLen): # Пока сумма индекса и окна меньше или равно чем число слов в выборке
    xBatch.append(wordIndices[index:index+xLen]) # Добавляем X в лист, что содержит наши фрагменты(окна) текста
    index += step # Смещаем наше индекс на наш шаг(step)

  return xBatch #  Лист фрагментов текста

In [ ]:
xLen = 256 # Длина каждого окна
step = 100 # шаг 
embeddingSize = 300 # Число измерений вектор-пространства для каждого нашего слова 

In [ ]:
X_train = getSetFromIndices(X_data[:-50000], xLen, step)
y_train = getSetFromIndices(y_data[:-50000], xLen, step)

In [ ]:
print('Длина xTrain:', len(X_train))
print('Длина yTrain:', len(y_train))

Длина xTrain: 9104
Длина yTrain: 9104


In [ ]:
X_test = getSetFromIndices(X_data[-50000:], xLen, step)
y_test = getSetFromIndices(y_data[-50000:], xLen, step)

In [ ]:
# Функция нужна для точности
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

In [ ]:
# Функция по созданию сети UNET
def create_unet(k = 1, num_classes = num_of_cat, input_shape= (30, 300)):
    img_input = Input(input_shape) 

    # Блок 1
    x = Conv1D(64 * k , 3, padding='same')(img_input) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k, 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    # Блок 2
    x = MaxPooling1D()(block_1_out)
    x = Conv1D(128 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    # Блок 3
    x = MaxPooling1D()(block_2_out)
    x = Conv1D(256 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # Блок 4
    x = MaxPooling1D()(block_3_out)
    x = Conv1D(512 * k, 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(256 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(128 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

In [ ]:
def pad_zeros(phrase, xLen = 256): # Функция для дополнением нулями
  while len(phrase) < xLen: # Пока сущестующий размер меньше чем требуемый 
    phrase.append([0] * embeddingSize) # Продолжаем добавлять элементы(нули) в фразу(zero-pad)  
  if len(phrase) > xLen: # В противном случае, если фраза больше чем  требуемый размер
    phrase = phrase[:xLen] # Берём только требуемое число элементов
  return phrase # Возвращаем обработанную фразу

# Создаём выборку
def getSets(model, senI, tagI):
  xVector = [] # Лист будет содержать ембеддинг представление на каждый индекс
  tmp = [] # Временный список
  for text in senI: # Проходимся по каждому лист-тексту
    tmp=[]
    for word in text: # Проходимся по кажому слову лист-текста
      try: # Если слово есть в модели (словарь не слишком мал), мы не получим ошибку
        tmp.append(model[word])
      except: # В противном случае пропустим слово
        pass

    xVector.append(pad_zeros(tmp, xLen))
  temp = np.array(xVector)
  return np.array(xVector, dtype = np.float32), np.array(tagI)

In [ ]:
# Подаём лист листов слов в word2vec для обучающей выборке
# size = embeddingSize - Размер ембеддинга
# window = 10 -  минимальное расстояния между эмбеддинг словами
# min_count = 1 - Игнорируем все слова с частотой меньше чем 1
# workers = 10 -  Число потоков на обучение эмбеддинга
# iter = 20 -  Число эпох на обучение эмбеддинга
# max_vocab_size = 1e5 -  Число слов в "словарном запасе" word2vec
curTime = time.time()

modelGENSIM = word2vec.Word2Vec(X_train + X_test, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10, max_vocab_size = 1e5)
print('Обучение GENSIM модели заняло: ', round(time.time() - curTime, 2), 'с.')

Обучение GENSIM модели заняло:  53.93 с.


In [ ]:
GENSIMtrainX, GENSIMtrainY = getSets(modelGENSIM, X_train, y_train)
GENSIMtestX, GENSIMtestY = getSets(modelGENSIM, X_test, y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [ ]:
model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
model_b_UNET.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 256, 64)      57664       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 64)      256         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 64)      0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
history = model_b_UNET.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=10, batch_size=64) # Feed the images divided among 2 classes to the model

Epoch 1/10
143/143 [==============================] - 9s 62ms/step - loss: 0.4637 - dice_coef: 0.7997 - val_loss: 0.2352 - val_dice_coef: 0.8775
Epoch 2/10
143/143 [==============================] - 8s 55ms/step - loss: 0.1788 - dice_coef: 0.8882 - val_loss: 0.1956 - val_dice_coef: 0.8964
Epoch 3/10
143/143 [==============================] - 8s 55ms/step - loss: 0.1498 - dice_coef: 0.8846 - val_loss: 0.1852 - val_dice_coef: 0.8701
Epoch 4/10
143/143 [==============================] - 8s 56ms/step - loss: 0.1297 - dice_coef: 0.8794 - val_loss: 0.1961 - val_dice_coef: 0.8698
Epoch 5/10
143/143 [==============================] - 8s 56ms/step - loss: 0.1116 - dice_coef: 0.8744 - val_loss: 0.2114 - val_dice_coef: 0.8494
Epoch 6/10
143/143 [==============================] - 8s 56ms/step - loss: 0.0949 - dice_coef: 0.8692 - val_loss: 0.2335 - val_dice_coef: 0.8665
Epoch 7/10
143/143 [==============================] - 8s 56ms/step - loss: 0.0797 - dice_coef: 0.8639 - val_loss: 0.2482 - val_dic

In [ ]:
model_b_UNET.predict(GENSIMtestX[:64])[0][:].shape

(256, 17)

In [ ]:
def recognizeSet(XX, YY, model, tags, length, value):
  correct_list = np.array([0] * 17) #Инициализируем массив правильных ответов в нули (сколько раз правильно определили класс)
  incorrect_list =  np.array([0] * 17)  #Инициализируем массив неправильных ответов в нули  (сколько раз неправильно определили класс)
  XX_array = XX
  YY_array = YY
  pred = model.predict(XX_array)
  pred[pred < value] = 0
  pred[pred > value] = 1

  for element in range(YY_array.shape[0]): # Проходим по всем примерам в батче
    for word in range(YY_array.shape[1]):  # Проходим по каждому слову
      for category in range(YY_array.shape[2]): # Проходим по каждой категории в слове
        if pred[element][word][category] == YY_array[element][word][category]: # Если предсказанное значение совпадает с истинным:
          correct_list[category] += 1 # Отмечаем, что мы правильно предсказали класс объекта
        else:  # Если предсказанное значение НЕ совпадает с истенным:
          incorrect_list[category] += 1 # Отмечаем, что мы не правильно предсказали класс объекта
      
  # Итоговая точность для каждого класса = кол. 100% * правильных/(кол. неправильных + кол. правильных)
  for i in range(17):
   print("Сеть распознала категорию  '{}' с точностью в {}%".format(tags[i], round(100*correct_list[i]/(correct_list[i] + incorrect_list[i]), 2)))
  total = round(100*np.mean(correct_list/(correct_list + incorrect_list)),2) 
  print("Средняя точность {}%".format(total))

In [ ]:
recognizeSet(GENSIMtestX, GENSIMtestY, model_b_UNET, list_of_tags, xLen, 0.1) # Проверяем UNET архитектуру

Сеть распознала категорию  'O' с точностью в 92.67%
Сеть распознала категорию  'B-geo' с точностью в 98.13%
Сеть распознала категорию  'B-gpe' с точностью в 99.55%
Сеть распознала категорию  'B-tim' с точностью в 99.28%
Сеть распознала категорию  'B-org' с точностью в 98.45%
Сеть распознала категорию  'I-geo' с точностью в 99.52%
Сеть распознала категорию  'B-per' с точностью в 99.0%
Сеть распознала категорию  'I-per' с точностью в 98.96%
Сеть распознала категорию  'I-org' с точностью в 98.84%
Сеть распознала категорию  'I-tim' с точностью в 99.57%
Сеть распознала категорию  'B-art' с точностью в 99.97%
Сеть распознала категорию  'I-art' с точностью в 99.98%
Сеть распознала категорию  'B-nat' с точностью в 99.99%
Сеть распознала категорию  'I-gpe' с точностью в 99.99%
Сеть распознала категорию  'I-nat' с точностью в 100.0%
Сеть распознала категорию  'B-eve' с точностью в 99.97%
Сеть распознала категорию  'I-eve' с точностью в 99.98%
Средняя точность 99.05%
